# Outline

* Implement and train a fully connected and a convolutional neural net using PyTorch
* We will also illustrate how the ConvNet makes use of specific assumptions

In [ ]:
% run plot_helper.py

In [ ]:
plt_style()

In [ ]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(10)

In [ ]:
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy

#helper function to count number of parameters in a neural neywrok
def get_n_params(model):
    np=0
    for p in list(model.parameters()):
        np += p.nelement()
    return np

# Load the Dataset (MNIST)


We can use PyTorch DataLoader utilities for this. This will download, shuffle, normalize data and arrange it in batches.

In [ ]:
input_size  = 28*28   # images are 28x28 pixels
output_size = 10      # there are 10 classes

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=1000, shuffle=True)

In [ ]:
# show some images
plt.figure()
for i in range(10):
    plt.subplot(2, 5, i + 1)
    image, _ = train_loader.dataset.__getitem__(i)
    plt.imshow(image.squeeze().numpy())

# Create the model classes

``nn.Module`` - Neural network module. Convenient way of encapsulating parameters, with helpers for moving them to GPU, exporting, loading, etc.

All our neural nets will inherit from this class.

In [ ]:
class FC2Layer(nn.Module):
    """A simple Fully Connected neural net class.
    
    In PyTorch, subclassing your model from torch.nn.Module
    takes care of low-level concerns for you such as defining
    the backward pass and keeping track of network parameters.
    
    Arguments
    ----------
    input_size : int
        The number of features in the input tensor.
    n_hidden : int
        The number of features in the hidden state of the FC layer.
    output_size : int
        The number of classes in the output tensor.
    """
    def __init__(self, input_size, n_hidden, output_size):
        super(FC2Layer, self).__init__()
        self.input_size = input_size
        self.linear1 = nn.Linear(input_size, n_hidden)
        self.linear2 = nn.Linear(n_hidden, n_hidden)
        self.linear3 = nn.Linear(n_hidden, output_size)

    def forward(self, x):
        #since the images are square, we need to flatten before 
        #feeding to the linear layer
        x = x.view(-1, self.input_size)
        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)
        x = F.relu(x)
        x = self.linear3(x)
        x = F.relu(x)
        x = F.log_softmax(x, dim=1)
        return x
    
    
class CNN(nn.Module):
    """A simple CNN class.
    
    In PyTorch, subclassing your model from torch.nn.Module
    takes care of low-level concerns for you such as defining
    the backward pass and keeping track of network parameters.
    
    Arguments
    ----------
    input_size : int
        The number of features in the input tensor.
    n_feature : int
        The number of output channels of the CNN, or the number of 
        feature maps produced at the output of the layer
    """
    def __init__(self, input_size, n_feature):
        super().__init__()
        self.n_feature = n_feature
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=n_feature, kernel_size=5)
        self.conv2 = nn.Conv2d(n_feature, n_feature, kernel_size=5)
        #the parameters below are manually tuned to get 
        #overall number of trainable parametres close to that in FC neural net
        self.fc1 = nn.Linear(n_feature*4*4, 50)
        self.fc2 = nn.Linear(50, 10)
        

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)
        x = x.view(-1, self.n_feature*4*4)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x
 


### Sequence model: alternative API

```python
network = nn.Sequential(
    nn.Linear(input_size, n_hidden), 
    nn.ReLU(), 
    nn.Linear(n_hidden, n_hidden), 
    nn.ReLU(), 
    nn.Linear(n_hidden, output_size), 
    nn.LogSoftmax(dim=1)
)
```        

## Running on a GPU: device string

Switching between CPU and GPU in PyTorch is controlled via a device string, which will seemlessly determine whether GPU is available, falling back to CPU if not:

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
accuracy_list = []

def train(epoch, model, train_loader, optimizer, device, perm=torch.arange(0, 784).long()):
    """Train a model to classify handwritten digits using FC neural net
    
    Parameters
    ----------
    model : torch.nn.Module
        The model to train.
    train_loader : The data generator instance used to produce training data.
    optimizer : torch.optim.Optimizer
        The optimization algorithm used to updated the network parameters.
    device : torch.device
        The device to which tensors will be moved.
    perm : torch.tensor
        Tensor defining a random permutation of indices, 
        the default leaves indices in original order  
    Returns
    -------
    tuple
        The number of correctly classified sequences and the loss, respectively.
    """
    
    # Set the model to training mode. This will turn on layers that would
    # otherwise behave differently during evaluation, such as dropout or batch normalization.
    model.train()
    
    # Iterate over every batch
    for batch_idx, (data, target) in enumerate(train_loader):
  
        # permute pixels
        data = data.view(-1, 28*28)
        data = data[:, perm]
        data = data.view(-1, 1, 28, 28).to(device)
            
        # For each new batch, clear the gradient buffers of the optimized parameters.
        # Otherwise, gradients from the previous batch would be accumulated.
        optimizer.zero_grad()
        
        # Perform the forward pass of the model.
        output = model(data)
        
        # Compute the value of the loss for this batch.
        loss = F.nll_loss(output, target)
        
        #Backpropagation step
        loss.backward()
        #Weight update
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
def test(model, test_loader, optimizer, device, perm=torch.arange(0, 784).long()):
    """Train a model to classify handwritten digits using FC neural net
    
    Parameters
    ----------
    model : torch.nn.Module
        The model to train.
    test_loader : The data generator instance used to produce test data.
    optimizer : torch.optim.Optimizer
        The optimization algorithm used to updated the network parameters.
    device : torch.device
        The device to which tensors will be moved.
    perm : torch.tensor
        Tensor defining a random permutation of indices, 
        the default leaves indices in original order  
    Returns
    -------
    tuple
        The number of correctly classified sequences and the loss, respectively.
    """
        
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        # permute pixels
        data = data.view(-1, 28*28)
        data = data[:, perm]
        data = data.view(-1, 1, 28, 28).to(device)
        
        #perform forward pass
        output = model(data)
        
        # sum up batch loss 
        test_loss += F.nll_loss(output, target, size_average=False).item()   
        
        # get the index of the max log-probability   
        pred = output.data.max(1, keepdim=True)[1]                                                               
        correct += pred.eq(target.data.view_as(pred)).cpu().sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    accuracy_list.append(accuracy)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        accuracy))

# Train a small fully-connected network

In [ ]:
n_hidden = 8    # number of hidden units

model = FC2Layer(input_size, n_hidden, output_size)
model.to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
print('Number of parameters: {}'.format(get_n_params(model)))

for epoch in range(0, 1):
    train(epoch, model, train_loader, optimizer, device)
    test(model, test_loader, optimizer, device)

# Train a ConvNet with the same number of parameters

In [ ]:
# Training settings 
n_features    = 6     # number of feature maps

model = CNN(input_size, n_features)
model.to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
print('Number of parameters: {}'.format(get_n_params(model)))

for epoch in range(0, 1):
    train(epoch, model, train_loader, optimizer, device)
    test(model, test_loader, optimizer, device)

# The ConvNet performs better with the same number of parameters, thanks to its use of prior knowledge about images

* Use of convolution: Locality and stationarity in images
* Pooling: builds in some translation invariance

# What happens if the assumptions are no longer true?


In [ ]:
perm = torch.randperm(784)
plt.figure()
for i in range(10):
    image, _ = train_loader.dataset.__getitem__(i)
    # permute pixels
    image_perm = image.view(-1, 28*28).clone()
    image_perm = image_perm[:, perm]
    image_perm = image_perm.view(-1, 1, 28, 28)
    plt.subplot(4, 5, i + 1)
    plt.imshow(image.squeeze().numpy())
    plt.axis('off')
    plt.subplot(4, 5, i + 11)
    plt.imshow(image_perm.squeeze().numpy())
    plt.axis('off')

# ConvNet with permuted pixels

In [ ]:
# Training settings 
n_features    = 6     # number of feature maps

model = CNN(input_size, n_features)
model.to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
print('Number of parameters: {}'.format(get_n_params(model)))

for epoch in range(0, 1):
    train(epoch, model, train_loader, optimizer, device, perm)
    test(model, test_loader, optimizer, device, perm)

# Fully-Connected with Permuted Pixels

In [ ]:
n_hidden    = 8    # number of hidden units

model = FC2Layer(input_size, n_hidden, output_size)
model.to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
print('Number of parameters: {}'.format(get_n_params(model)))

for epoch in range(0, 1):
    train(epoch, model, train_loader, optimizer, device, perm)
    test(model, test_loader, optimizer, device, perm)

# The ConvNet's performance drops when we permute the pixels, but the Fully-Connected Network's performance stays the same

* ConvNet makes the assumption that pixels lie on a grid and are stationary/local
* It loses performance when this assumption is wrong
* The fully-connected network does not make this assumption
* It does less well when it is true, since it doesn't take advantage of this prior knowledge
* But it doesn't suffer when the assumption is wrong

In [ ]:
plt.bar(('NN image', 'CNN image',
         'CNN scrambled', 'NN scrambled'),
        accuracy_list, width=0.4)
plt.ylim((min(accuracy_list)-5, 96))
plt.ylabel('Accuracy [%]')
for tick in plt.gca().xaxis.get_major_ticks():
    tick.label.set_fontsize(20)
plt.title('Performance comparison');